In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import word2vec
from konlpy.tag import Okt
from tqdm import tqdm
import re
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = pd.read_csv('data/전체웹툰_제목,장르,시놉시스,태그.csv', encoding='utf-8')
data

,title,genre,synopsis,tag
0,참교육,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!<부활남> 채...,"블루스트링, 사이다, 학원물, 다크히어로, 먼치킨, 액션"
1,뷰티풀 군바리,드라마,'여자도 군대에 간다면?'본격 여자도 군대 가는 만화!,"직업드라마, 밀리터리, 드라마"
2,퀘스트지상주의,드라마,"[외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하는 작품!공부, 싸움,...","게임판타지, 학원물, 드라마"
3,장씨세가 호위무사,무협/사극,‘당신이 부른 것이오. 나란 사람을... ’은둔고수 광휘. 호위무사 되다.웹소설 원...,"사이다, 힘숨찐, 판무, 먼치킨, 무협/사극, 소설원작"
4,윈드브레이커,스포츠,혼자서 자전거를 즐겨타던 모범생 조자현.원치 않게 자전거 크루의 일에 자꾸 휘말리게...,"스포츠성장, 스포츠"
...,...,...,...,...
2002,AA,"스토리, 로맨스",대머리 사절을 외치는 여인과 대머리 일것 같은 그녀의 이상형이 벌이는 로맨틱 코미디...,"로맨스, 완결무료, 완결로맨스"
2003,사랑in,"옴니버스, 로맨스",전세훈의 감성터치-사랑in 우리들의 이야기가 수채화 물감처럼 잔잔한 감동으로. 잃어...,"로맨스, 완결로맨스"
2004,까뱅,"에피소드, 개그",치킨시티의 새로운 영웅 '까뱅' 오늘도 그의 활약은 계속된다!,"개그, 완결무료, 완결개그"
2005,와탕카,"에피소드, 개그","와탕카='만세','지화자' 좋은 패가 들어오면 외치는 단어 와탕카! 우주인 작가님의...","개그, 완결무료, 완결개그"


In [3]:
data['sentence'] = data.apply(lambda row: ' '.join(row.values.astype(str)),axis=1)
data['sentence']

0       참교육 액션 무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!...
1       뷰티풀 군바리 드라마 '여자도 군대에 간다면?'본격 여자도 군대 가는 만화! 직업드...
2       퀘스트지상주의 드라마 [외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하...
3       장씨세가 호위무사 무협/사극 ‘당신이 부른 것이오. 나란 사람을... ’은둔고수 광...
4       윈드브레이커 스포츠 혼자서 자전거를 즐겨타던 모범생 조자현.원치 않게 자전거 크루의...
                              ...                        
2002    AA 스토리, 로맨스 대머리 사절을 외치는 여인과 대머리 일것 같은 그녀의 이상형이...
2003    사랑in 옴니버스, 로맨스 전세훈의 감성터치-사랑in 우리들의 이야기가 수채화 물감...
2004    까뱅 에피소드, 개그 치킨시티의 새로운 영웅 '까뱅' 오늘도 그의 활약은 계속된다!...
2005    와탕카 에피소드, 개그 와탕카='만세','지화자' 좋은 패가 들어오면 외치는 단어 ...
2006    타투 옴니버스, 드라마 밝고 쾌활한 여고생 현진과 새로 오픈한 타투 가게의 진우, ...
Name: sentence, Length: 2007, dtype: object

In [4]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    # # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [5]:
%time sentences=data['sentence'].apply(preprocessing)

Wall time: 24.7 ms


In [6]:
sentences

0       참교육 액션 무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다 ...
1       뷰티풀 군바리 드라마  여자도 군대에 간다면  본격 여자도 군대 가는 만화  직업드...
2       퀘스트지상주의 드라마  외모지상주의    싸움독학    인생존망 과 세계관을 공유하...
3       장씨세가 호위무사 무협 사극  당신이 부른 것이오  나란 사람을     은둔고수 광...
4       윈드브레이커 스포츠 혼자서 자전거를 즐겨타던 모범생 조자현 원치 않게 자전거 크루의...
                              ...                        
2002    AA 스토리  로맨스 대머리 사절을 외치는 여인과 대머리 일것 같은 그녀의 이상형이...
2003    사랑in 옴니버스  로맨스 전세훈의 감성터치 사랑in 우리들의 이야기가 수채화 물감...
2004    까뱅 에피소드  개그 치킨시티의 새로운 영웅  까뱅  오늘도 그의 활약은 계속된다 ...
2005    와탕카 에피소드  개그 와탕카  만세   지화자  좋은 패가 들어오면 외치는 단어 ...
2006    타투 옴니버스  드라마 밝고 쾌활한 여고생 현진과 새로 오픈한 타투 가게의 진우  ...
Name: sentence, Length: 2007, dtype: object

In [7]:
okt = Okt()

nouns = [okt.nouns(sentence) for sentence in sentences]
nouns

[['교육',
  '액션',
  '교권',
  '위해',
  '교권',
  '보호국',
  '소속',
  '나화',
  '진의',
  '교육',
  '시작',
  '부활',
  '남',
  '채용',
  '택',
  '작가',
  '기녀',
  '한가람',
  '작가',
  '신작',
  '블루스',
  '트링',
  '사이다',
  '학원물',
  '다크',
  '히어로',
  '먼치킨',
  '액션'],
 ['뷰티풀',
  '군',
  '바리',
  '드라마',
  '여자도',
  '군대',
  '간다',
  '본격',
  '여자도',
  '군대',
  '만화',
  '직업',
  '드라마',
  '밀리터리',
  '드라마'],
 ['퀘스트',
  '지상',
  '주의',
  '드라마',
  '외모',
  '지상',
  '주의',
  '싸움',
  '독학',
  '인생',
  '존망',
  '과',
  '세계관',
  '공유',
  '작품',
  '공부',
  '싸움',
  '외모',
  '뭐',
  '하나',
  '것',
  '고등학생',
  '김수현',
  '의',
  '눈앞',
  '갑자기',
  '퀘스트',
  '창',
  '첫',
  '퀘스트',
  '엄마',
  '사과',
  '를',
  '완료',
  '수현',
  '보상',
  '키',
  '퀘스트',
  '퀘스트',
  '완료',
  '다음',
  '퀘스트',
  '학교',
  '일진',
  '녀',
  '키스',
  '게임판타지',
  '학원물',
  '드라마'],
 ['장씨',
  '세',
  '호위',
  '무사',
  '무협',
  '사극',
  '당신',
  '것',
  '이오',
  '사람',
  '은둔',
  '고수',
  '광휘',
  '호위',
  '무사',
  '웹소설',
  '원작',
  '웰',
  '메이드',
  '무협',
  '시대극',
  '사이다',
  '힘숨',
  '판무',
  '먼치킨',
  '무협',
  '사극',
  '소설',
  '원작'],
 ['윈드브

In [8]:
import gensim

model = gensim.models.Word2Vec.load('data/ko/ko.bin')

In [9]:
model.wv.save_word2vec_format("ko.bin.gz", binary = False)

In [10]:
model2 = word2vec.Word2Vec(sentences=nouns, size=200, window=4, hs=1,
                           min_count=2, sg=1, workers=2)
# size = 200차원 벡터로 / window = 주변단어 앞뒤로 4개까지/ workers = 코어 2
#min_count = 출현빈도 2번 미만인건 분석제외/ sg = Skip-Gram 분석방법론 사용
total_examples = model2.corpus_count
total_examples

2007

In [12]:
model2.build_vocab([list(model.wv.vocab.keys())], update=True)
model2.intersect_word2vec_format("ko.bin.gz", binary=False)

In [13]:
model2.train(nouns,total_examples=total_examples,epochs=model2.iter)

C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(229393, 293410)

In [52]:
model2.wv.most_similar("영웅")

[('악당', 0.5823014974594116),
 ('여신', 0.5505523681640625),
 ('비극', 0.5426483154296875),
 ('용사', 0.5414282083511353),
 ('요정', 0.5367072224617004),
 ('서사시', 0.5364639163017273),
 ('주인공', 0.5148755311965942),
 ('악마', 0.5130380392074585),
 ('마법사', 0.5130000114440918),
 ('여인', 0.4826076328754425)]

In [2]:
model2.save("ko_webtoon.bin")

NameError: name 'model2' is not defined

## 장르 수정

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import word2vec
from konlpy.tag import Okt
from tqdm import tqdm
import re
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
data = pd.read_csv('data/전체웹툰_장르수정.csv', encoding='utf-8')
data

,title,genre,synopsis,tag
0,참교육,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!<부활남> 채...,"블루스트링, 사이다, 학원물, 다크히어로, 먼치킨, 액션"
1,뷰티풀 군바리,드라마,'여자도 군대에 간다면?'본격 여자도 군대 가는 만화!,"직업드라마, 밀리터리, 드라마"
2,퀘스트지상주의,드라마,"[외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하는 작품!공부, 싸움,...","게임판타지, 학원물, 드라마"
3,장씨세가 호위무사,무협/사극,‘당신이 부른 것이오. 나란 사람을... ’은둔고수 광휘. 호위무사 되다.웹소설 원...,"사이다, 힘숨찐, 판무, 먼치킨, 무협/사극, 소설원작"
4,윈드브레이커,스포츠,혼자서 자전거를 즐겨타던 모범생 조자현.원치 않게 자전거 크루의 일에 자꾸 휘말리게...,"스포츠성장, 스포츠"
...,...,...,...,...
2002,AA,로맨스,대머리 사절을 외치는 여인과 대머리 일것 같은 그녀의 이상형이 벌이는 로맨틱 코미디...,"로맨스, 완결무료, 완결로맨스"
2003,사랑in,로맨스,전세훈의 감성터치-사랑in 우리들의 이야기가 수채화 물감처럼 잔잔한 감동으로. 잃어...,"로맨스, 완결로맨스"
2004,까뱅,개그,치킨시티의 새로운 영웅 '까뱅' 오늘도 그의 활약은 계속된다!,"개그, 완결무료, 완결개그"
2005,와탕카,개그,"와탕카='만세','지화자' 좋은 패가 들어오면 외치는 단어 와탕카! 우주인 작가님의...","개그, 완결무료, 완결개그"


In [5]:
data['sentence'] = data.apply(lambda row: ' '.join(row.values.astype(str)),axis=1)
data['sentence']

0       참교육 액션 무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!...
1       뷰티풀 군바리 드라마 '여자도 군대에 간다면?'본격 여자도 군대 가는 만화! 직업드...
2       퀘스트지상주의 드라마 [외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하...
3       장씨세가 호위무사 무협/사극 ‘당신이 부른 것이오. 나란 사람을... ’은둔고수 광...
4       윈드브레이커 스포츠 혼자서 자전거를 즐겨타던 모범생 조자현.원치 않게 자전거 크루의...
                              ...                        
2002    AA  로맨스 대머리 사절을 외치는 여인과 대머리 일것 같은 그녀의 이상형이 벌이는...
2003    사랑in  로맨스 전세훈의 감성터치-사랑in 우리들의 이야기가 수채화 물감처럼 잔잔...
2004    까뱅  개그 치킨시티의 새로운 영웅 '까뱅' 오늘도 그의 활약은 계속된다! 개그, ...
2005    와탕카  개그 와탕카='만세','지화자' 좋은 패가 들어오면 외치는 단어 와탕카! ...
2006    타투  드라마 밝고 쾌활한 여고생 현진과 새로 오픈한 타투 가게의 진우, 유기를 중...
Name: sentence, Length: 2007, dtype: object

In [6]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    # # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [7]:
%time sentences=data['sentence'].apply(preprocessing)

Wall time: 21.9 ms


In [8]:
okt = Okt()

nouns = [okt.nouns(sentence) for sentence in sentences]
nouns

[['교육',
  '액션',
  '교권',
  '위해',
  '교권',
  '보호국',
  '소속',
  '나화',
  '진의',
  '교육',
  '시작',
  '부활',
  '남',
  '채용',
  '택',
  '작가',
  '기녀',
  '한가람',
  '작가',
  '신작',
  '블루스',
  '트링',
  '사이다',
  '학원물',
  '다크',
  '히어로',
  '먼치킨',
  '액션'],
 ['뷰티풀',
  '군',
  '바리',
  '드라마',
  '여자도',
  '군대',
  '간다',
  '본격',
  '여자도',
  '군대',
  '만화',
  '직업',
  '드라마',
  '밀리터리',
  '드라마'],
 ['퀘스트',
  '지상',
  '주의',
  '드라마',
  '외모',
  '지상',
  '주의',
  '싸움',
  '독학',
  '인생',
  '존망',
  '과',
  '세계관',
  '공유',
  '작품',
  '공부',
  '싸움',
  '외모',
  '뭐',
  '하나',
  '것',
  '고등학생',
  '김수현',
  '의',
  '눈앞',
  '갑자기',
  '퀘스트',
  '창',
  '첫',
  '퀘스트',
  '엄마',
  '사과',
  '를',
  '완료',
  '수현',
  '보상',
  '키',
  '퀘스트',
  '퀘스트',
  '완료',
  '다음',
  '퀘스트',
  '학교',
  '일진',
  '녀',
  '키스',
  '게임판타지',
  '학원물',
  '드라마'],
 ['장씨',
  '세',
  '호위',
  '무사',
  '무협',
  '사극',
  '당신',
  '것',
  '이오',
  '사람',
  '은둔',
  '고수',
  '광휘',
  '호위',
  '무사',
  '웹소설',
  '원작',
  '웰',
  '메이드',
  '무협',
  '시대극',
  '사이다',
  '힘숨',
  '판무',
  '먼치킨',
  '무협',
  '사극',
  '소설',
  '원작'],
 ['윈드브

In [9]:
import gensim

model = gensim.models.Word2Vec.load('data/ko/ko.bin')

In [10]:
model.wv.save_word2vec_format("ko.bin.gz", binary = False)

In [11]:
model2 = word2vec.Word2Vec(sentences=nouns, size=200, window=4, hs=1,
                           min_count=2, sg=1, workers=2)
# size = 200차원 벡터로 / window = 주변단어 앞뒤로 4개까지/ workers = 코어 2
#min_count = 출현빈도 2번 미만인건 분석제외/ sg = Skip-Gram 분석방법론 사용
total_examples = model2.corpus_count
total_examples

2007

In [12]:
model2.build_vocab([list(model.wv.vocab.keys())], update=True)
model2.intersect_word2vec_format("ko.bin.gz", binary=False)

In [13]:
model2.train(nouns,total_examples=total_examples,epochs=model2.iter)

C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(226130, 286180)

In [15]:
model2.wv.most_similar("스토리")

[('캐릭터', 0.548080325126648),
 ('판타지', 0.5304969549179077),
 ('코미디', 0.5209457278251648),
 ('러브', 0.5146316885948181),
 ('코믹', 0.5092445015907288),
 ('시리즈', 0.498371422290802),
 ('게임', 0.4972790777683258),
 ('어드벤처', 0.49619531631469727),
 ('에피소드', 0.4928416609764099),
 ('엔딩', 0.491845041513443)]

In [16]:
model2.save("ko_webtoon_word2vec.bin")